In [1]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from task_cls import (
    PoliceCallBasicInfo,
    SafetySuggestionProvided
)

from voice_agent_flow.node import AgentNode
from voice_agent_flow.llms import create_pydantic_azure_openai
from voice_agent_flow.llms import create_ollama_model

from voice_agent_flow.runner import AgentRunner

model = create_pydantic_azure_openai('gpt-4o-mini')

instruction = """
You are a police call center agent(working at 110). You task is to talk with caller via telephone to collection information.
You resopnse should be berief and direct to the point.
"""


def location_reachable_on_map(location: str) -> bool:
    """Search if the location is reachable on map."""
    print("Location is reachable on map tool called with location:", location)
    return True

agents = {
    "police_call_basic_info": AgentNode(
        name="police_call_basic_info",
        model=model,
        instruction=instruction,
        task_cls= PoliceCallBasicInfo,
        step_instruction=(
            "Collect basic information about the police call including case location, case type, description and caller name.) "
            "ask the question one at a time, do not ask multiple questions in one message."
            "When caller told you the location, make sure to call the tool(location_reachable_on_map) to check if the location is reachable on map."
            "If the location is not reachable, ask the caller to provide more details about the location."
        ),
        examples=["请问您遇到什么紧急情况？ / 发生在哪里？/ 能简单描述一下吗？/ 您的姓名是？"],
        tools = [location_reachable_on_map],
        ),
    "safety_suggestion": AgentNode(
        name="safety_suggestion",
        model=model,
        instruction=instruction,
        task_cls= SafetySuggestionProvided,
        step_instruction="Based on the collected information, provide safety suggestion(A clear command to keep safe) to the caller. After the caller responded to your suggestion, create the schema.",
        examples=["请您保持冷静，.......(provide clear safety suggestion)", "请您打开窗户，保持空气流通"],
        ),
    }

runner = AgentRunner(
    agents=agents, 
    entry_agent_name="police_call_basic_info",
    ending_message="好的，我们这就派人过去处理您的情况，请您保持电话畅通！"
)  
       
     

In [3]:
runner.run("喂，你要，我要报警")

'请问您遇到什么紧急情况？'

In [4]:
runner.run("有人在外面撬我家门锁")

'哦，发生在哪里？请告诉我具体的地址或地点。'

In [5]:
runner.run("背景海淀区天地大厦")

Location is reachable on map tool called with location: 海淀区天地大厦


'好的，您所说的地点是 reachable 的。请问这是什么类型的案件？'

In [6]:
runner.run("就是这样，一直在敲，哐哐哐")

'我明白了，请您简单描述一下案件的具体情况。'

In [7]:
runner.run("我叫王小明")

Collected information: {'case_location': '海淀区天地大厦', 'case_type': '撬门', 'description': '有人在外面撬我家门锁，持续敲门，声音很大。', 'caller_name': '王小明'}
Transfer to safety_suggestion


'好的，王小明。为了保障您的安全，请您立即锁好门，保持冷静，并寻找一个安全的地方躲避，直到警方到达。请确认您是否已采取行动。'

In [8]:
runner.run("好的")

Collected information: {'suggestion_provided': True}
All tasks completed.


'好的，我们这就派人过去处理您的情况，请您保持电话畅通！'

In [9]:
runner.show_information()

[{'case_location': '海淀区天地大厦',
  'case_type': '撬门',
  'description': '有人在外面撬我家门锁，持续敲门，声音很大。',
  'caller_name': '王小明'},
 {'suggestion_provided': True}]